### Kelas I1 Kelompok 7
#### Content Based Recommender System (Kasus Rekomendasi Film)
1. Batara Agung Pramudito Trilaksono (082011633055)
2. Fikri Yoma Rosyidan (082011633076)
3. Aristo Riza Muhammad (082011633077)
4. Rais Ilham Nustara (082011633090)
5. Aida Filza Binti Aidrus (502122060005)

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
data = pd.read_csv('movie_metadata.csv')
data.describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
count,4993.000000,5028.000000,4939.000000,5020.000000,5036.000000,4.159000e+03,5.043000e+03,5043.000000,5030.000000,5022.000000,4.551000e+03,4935.000000,5030.000000,5043.000000,4714.000000,5043.000000
mean,140.194272,107.201074,686.509212,645.009761,6560.047061,4.846841e+07,8.366816e+04,9699.063851,1.371173,272.770808,3.975262e+07,2002.470517,1651.754473,6.442138,2.220403,7525.964505
std,121.601675,25.197441,2813.328607,1665.041728,15020.759120,6.845299e+07,1.384853e+05,18163.799124,2.013576,377.982886,2.061149e+08,12.474599,4042.438863,1.125116,1.385113,19320.445110
min,1.000000,7.000000,0.000000,0.000000,0.000000,1.620000e+02,5.000000e+00,0.000000,0.000000,1.000000,2.180000e+02,1916.000000,0.000000,1.600000,1.180000,0.000000
25%,50.000000,93.000000,7.000000,133.000000,614.000000,5.340988e+06,8.593500e+03,1411.000000,0.000000,65.000000,6.000000e+06,1999.000000,281.000000,5.800000,1.850000,0.000000
50%,110.000000,103.000000,49.000000,371.500000,988.000000,2.551750e+07,3.435900e+04,3090.000000,1.000000,156.000000,2.000000e+07,2005.000000,595.000000,6.600000,2.350000,166.000000
75%,195.000000,118.000000,194.500000,636.000000,11000.000000,6.230944e+07,9.630900e+04,13756.500000,2.000000,326.000000,4.500000e+07,2011.000000,918.000000,7.200000,2.350000,3000.000000
max,813.000000,511.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,1.221550e+10,2016.000000,137000.000000,9.500000,16.000000,349000.000000


In [3]:
# data yang digunakan
data = data[['movie_title','num_critic_for_reviews', 'duration', 'gross', 'budget','title_year', 'imdb_score']]
data.isna().sum()

movie_title                 0
num_critic_for_reviews     50
duration                   15
gross                     884
budget                    492
title_year                108
imdb_score                  0
dtype: int64

In [4]:
# Menghuapus data duplikat
data_no_duplicates = data.drop_duplicates()
data = data_no_duplicates
data_no_duplicates = data.drop_duplicates(subset=['movie_title', 'title_year'])
data = data_no_duplicates

In [5]:
# mengapus spasi diawal dan diakhir judul
data['movie_title'] = data['movie_title'].str.strip()

In [6]:
# handling missing value
data.fillna(method="bfill",inplace=True)

In [7]:
data.head(10)

,movie_title,num_critic_for_reviews,duration,gross,budget,title_year,imdb_score
0,Avatar,723.0,178.0,760505847.0,237000000.0,2009.0,7.9
1,Pirates of the Caribbean: At World's End,302.0,169.0,309404152.0,300000000.0,2007.0,7.1
2,Spectre,602.0,148.0,200074175.0,245000000.0,2015.0,6.8
3,The Dark Knight Rises,813.0,164.0,448130642.0,250000000.0,2012.0,8.5
4,Star Wars: Episode VII - The Force Awakens,462.0,132.0,73058679.0,263700000.0,2012.0,7.1
5,John Carter,462.0,132.0,73058679.0,263700000.0,2012.0,6.6
6,Spider-Man 3,392.0,156.0,336530303.0,258000000.0,2007.0,6.2
7,Tangled,324.0,100.0,200807262.0,260000000.0,2010.0,7.8
8,Avengers: Age of Ultron,635.0,141.0,458991599.0,250000000.0,2015.0,7.5
9,Harry Potter and the Half-Blood Prince,375.0,153.0,301956980.0,250000000.0,2009.0,7.5


### Membuat Recommender System

In [8]:
movie_features = data.loc[:, "num_critic_for_reviews":].copy()
movie_features.head()

,num_critic_for_reviews,duration,gross,budget,title_year,imdb_score
0,723.0,178.0,760505847.0,237000000.0,2009.0,7.9
1,302.0,169.0,309404152.0,300000000.0,2007.0,7.1
2,602.0,148.0,200074175.0,245000000.0,2015.0,6.8
3,813.0,164.0,448130642.0,250000000.0,2012.0,8.5
4,462.0,132.0,73058679.0,263700000.0,2012.0,7.1


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(movie_features.values, movie_features.values)

In [10]:
# nilai matriks cosine similarity 
cosine_sim

array([[1.        , 0.89253031, 0.83431836, ..., 0.96824449, 0.96306654,
        0.95820985],
       [0.89253031, 1.        , 0.99327609, ..., 0.78975726, 0.77281149,
        0.72665644],
       [0.83431836, 0.99327609, 1.        , ..., 0.71684148, 0.69745632,
        0.6422833 ],
       ...,
       [0.96824449, 0.78975726, 0.71684148, ..., 1.        , 0.99961049,
        0.97925231],
       [0.96306654, 0.77281149, 0.69745632, ..., 0.99961049, 1.        ,
        0.98209637],
       [0.95820985, 0.72665644, 0.6422833 , ..., 0.97925231, 0.98209637,
        1.        ]])

In [11]:
cosine_sim.shape

(4919, 4919)

In [12]:
# memberi nomor index
index = pd.Series(data.index, index=data['movie_title'])

In [13]:
index.head(11)

movie_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
Star Wars: Episode VII - The Force Awakens     4
John Carter                                    5
Spider-Man 3                                   6
Tangled                                        7
Avengers: Age of Ultron                        8
Harry Potter and the Half-Blood Prince         9
Batman v Superman: Dawn of Justice            10
dtype: int64

In [14]:
def get_recommendation(movie_title, similarity=cosine_sim):
    
    idx = index[movie_title]
    
    # Get the pairwsie similarity scores of all movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movie based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movie
    sim_scores = sim_scores[0:10]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
        
    # Return the top 20 most similar movie
    result = data[['movie_title']].iloc[movie_indices]
    return result

In [15]:
# Mencetak 20 Film rekomendasi yang mirip dengan Iron Man
print('Film yang mirip dengan Hulk')
get_recommendation('Hulk')

Film yang mirip dengan Hulk


,movie_title
168,G.I. Joe: Retaliation
2673,The Marine
983,The Judge
3503,Bride & Prejudice
432,Jumper
1949,The Losers
1252,Autumn in New York
761,Joy
2245,Georgia Rule
1959,The Box
